In [95]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns 

pd.set_option('display.max_columns', 25)
pd.set_option('display.max_rows', 50)

#Random seed

np.random.seed(3301)
db_location = "dataset.csv"
data = pd.read_csv(db_location, delimiter=";")

In [96]:
# Dimensiones de los datos
rows, cols = data.shape

print(f"filas: {rows}, columnas: {cols}")

filas: 768, columnas: 11


In [97]:
# Demostración de los datos para claridad
display(data.sample(5))

,Hair color,Pregnancies,Glucose,City,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
235,Black,4,171,New York,72,0,0,436,479,26,1
295,Black,6,151,New York,62,31,120,355,692,28,0
144,Black,4,154,New York,62,31,284,328,237,23,0
292,Black,2,128,New York,78,37,182,433,1224,31,1
60,Black,2,84,New York,0,0,0,0,304,21,0


In [98]:
# Calidad de los datos
# Por facilidad de manipulación y estandarización, los nombres de columnas que estan en PascalCase o separados por espacio 
# como el Hair color serán pasados a snake_case.

snake_case_names = {"Hair color": "hair_color",
                    "Pregnancies": "pregnancies",
                    "Glucose": "glucose",
                    "BloodPressure": "blood_pressure",
                    "City": "city",
                    "SkinThickness": "skin_thickness",
                    "Insulin": "insulin",
                    "DiabetesPedigreeFunction": "diabetes_pedigree_function",
                    "Age": "age",
                    "Outcome": "outcome"}
data = data.rename(columns=snake_case_names)


data.dtypes

hair_color                    object
pregnancies                   object
glucose                       object
city                          object
blood_pressure                object
skin_thickness                object
insulin                       object
BMI                            int64
diabetes_pedigree_function    object
age                            int64
outcome                       object
dtype: object

Como se puede ver, tenemos solamente columnas de tipo string (corresponden a tipo object en pandas), e int (corresponde a int64 de numpy).

Sin embargo, las columnas de: Pregnancies, Glucose, BloodPressure, SkinThickness, Insulin y DiabetesPedigreeFunction registran un tipo object aunque su contenido en realidad es int. Esto indica que tienen un placeholder string para cuando no se registró un valor. Debemos convertirlas a int, teniendo en cuenta la conversión de los placeholders a NaN.

Primero convertimos dichas columnas, que deberían ser numéricas, a float64. Los datos que sean placeholders en string se convertiran en NaN. Luego nos desharemos de las columnas con datos que sean NaN para garantizar la calidad de los datos.

In [99]:
# Conversión de columnas mencionadas anteriormente a float64, usando coerción de errores (datos no numéricos)
headers = ["pregnancies", "glucose", "blood_pressure", "skin_thickness", "insulin", "diabetes_pedigree_function"]
for header in headers:
    data[header] = pd.to_numeric(data[header], errors="coerce")

In [100]:
# Eliminar filas con valores NaN para garantizar calidad de datos.

data = data.dropna()

# Mostrar tipos para ver que se hayan actualizado
data.dtypes

hair_color                     object
pregnancies                   float64
glucose                       float64
city                           object
blood_pressure                float64
skin_thickness                float64
insulin                       float64
BMI                             int64
diabetes_pedigree_function    float64
age                             int64
outcome                        object
dtype: object

In [101]:
## Descripción de los datos numéricos

data.describe()

,pregnancies,glucose,blood_pressure,skin_thickness,insulin,BMI,diabetes_pedigree_function,age
count,767.000000,767.000000,767.000000,767.000000,767.000000,767.000000,767.000000,767.000000
mean,3.839635,120.921773,69.096480,20.563233,79.903520,289.670143,432.395046,38.006519
std,3.368429,31.984561,19.366833,15.945349,115.283105,116.780873,336.144934,117.902397
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,21.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,251.500000,205.500000,24.000000
50%,3.000000,117.000000,72.000000,23.000000,32.000000,309.000000,337.000000,29.000000
75%,6.000000,140.500000,80.000000,32.000000,127.500000,359.000000,592.000000,41.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,671.000000,2329.000000,3256.000000


### Evaluando los datos no-numéricos


In [102]:
string_columns = ["hair_color", "city"]

only_string_data = data[string_columns]

In [104]:
for col in only_string_data:
    print(f"unique values of {col}: {only_string_data[col].unique()}")
    
data = data.drop(columns=["city"])

unique values of hair_color: ['Red' 'Black' 'Blue']
unique values of city: ['New York']


Podemos concluir que estos datos no necesitan limpieza dado que no contienen valores de placeholder. 

Removemos la columna de "City", dado que contiene un único valor y no será de utilidad al crear ninguno de los modelos porque esta columna no aporta información al modelo. 

In [105]:
data

,hair_color,pregnancies,glucose,blood_pressure,skin_thickness,insulin,BMI,diabetes_pedigree_function,age,outcome
0,Red,6.0,148.0,72.0,35.0,0.0,336,627.0,50,1
1,Black,1.0,85.0,66.0,29.0,0.0,266,351.0,31,0
2,Red,8.0,183.0,64.0,0.0,0.0,233,672.0,32,1
3,Black,1.0,89.0,66.0,23.0,94.0,281,167.0,21,0
4,Black,0.0,137.0,40.0,35.0,168.0,431,2288.0,33,1
...,...,...,...,...,...,...,...,...,...,...
763,Black,10.0,101.0,76.0,48.0,180.0,329,171.0,63,0
764,Black,2.0,122.0,70.0,27.0,0.0,368,34.0,27,0
765,Red,5.0,121.0,72.0,23.0,112.0,262,245.0,30,0
766,Black,1.0,126.0,60.0,0.0,0.0,301,349.0,47,1


In [107]:
# Observamos la distribución del color de pelo, nuesto único dato categórico, según la cantidad de entradas.
# Usar la funcion value_counts() para contar el numero de veces que aparece un elemento.
data.hair_color.value_counts().to_frame().head(15)

,hair_color
Black,685
Red,50
Blue,32


Vemos que la cantidad de datos por color de pelo es desproporcionada a favor de una de las variables. 
Pareciera además que esta variable tampoco tiene sentido con la investigación acerca de la diabetes, 
por lo que podría eliminarse del dataset. (PREGUNTAR)